<a href="https://colab.research.google.com/github/AndromedaOMA/SemEval-2026_Task4-Narrative-Similarity/blob/main/AllenNLP_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Acesta este codul prototip pentru Course of Action

In [ ]:
%%bash
set -e

curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba


bin/micromamba


In [ ]:
%%bash
set -e

./bin/micromamba create -y -n allennlp38 -c conda-forge python=3.8 pip
./bin/micromamba run -n allennlp38 pip install -q "torch==1.12.1" "allennlp==2.10.1" "allennlp-models==2.10.1"



Transaction

  Prefix: /root/.local/share/mamba/envs/allennlp38

  Updating specs:

   - python=3.8
   - pip


  Package              Version  Build                 Channel          Size
─────────────────────────────────────────────────────────────────────────────
  Install:
─────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex          0.1  conda_forge           conda-forge       3kB
  + _openmp_mutex          4.5  2_gnu                 conda-forge      24kB
  + bzip2                1.0.8  hda65f42_8            conda-forge     260kB
  + ca-certificates   2026.1.4  hbd8a1cb_0            conda-forge     147kB
  + icu                   78.2  h33c6efd_0            conda-forge      13MB
  + ld_impl_linux-64      2.45  default_hbd61a6d_105  conda-forge     731kB
  + libffi               3.5.2  h9ec8514_0            conda-forge      58kB
  + libgcc              15.2.0  he0feb66_16           conda-forge       1MB
  + libgcc-ng           15.2.0  h69

In [ ]:
%%bash
set -e

./bin/micromamba run -n allennlp38 bash -lc '
set -e
export LD_LIBRARY_PATH="$CONDA_PREFIX/lib:$LD_LIBRARY_PATH"
export LD_PRELOAD="$CONDA_PREFIX/lib/libstdc++.so.6:$CONDA_PREFIX/lib/libgcc_s.so.1"
export PYTHONUNBUFFERED=1

python -u - <<PY
import json, re, sys, time
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

def log(msg):
    print(msg, file=sys.stderr, flush=True)

SRL_URL = "https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz"
srl = Predictor.from_path(SRL_URL, cuda_device=-1)

ARG_RE = re.compile(r"\[([A-Z0-9]+): ([^\]]+)\]")

def split_sents(text):
    text = " ".join(str(text).split())
    if not text:
        return []
    return [p.strip() for p in re.split(r"(?<=[\.\!\?\;])\s+", text) if p.strip()]

def parse(desc):
    return {k: v.strip() for k, v in ARG_RE.findall(desc)}

STOP_VERBS = {
    "be","is","are","was","were","been","being",
    "have","has","had","do","does","did",
    "will","would","can","could","should","may","might","must"
}

def srl_events(text, keep=("ARG0","ARG1","ARG2"), min_args=1):
    events = []
    for sent in split_sents(text):
        out = srl.predict(sentence=sent)
        for fr in out.get("verbs", []):
            d = parse(fr.get("description",""))
            verb = (fr.get("verb") or d.get("V","")).strip().lower()
            if not verb or verb in STOP_VERBS:
                continue

            args = [d[a].lower() for a in keep if a in d]
            if len(args) < min_args:
                continue

            events.append(" | ".join([verb] + args))
    return events

input_path  = "test_track_a.jsonl"
output_path = "events_test.jsonl"

with open(input_path, "r", encoding="utf-8") as fin, \
     open(output_path, "w", encoding="utf-8") as fout:

    for line in fin:
        line = line.strip()
        if not line:
            continue

        ex = json.loads(line)
        out = {
            "anchor_events": srl_events(ex.get("anchor_text","")),
            "A_events": srl_events(ex.get("text_a","")),
            "B_events": srl_events(ex.get("text_b","")),
            label": bool(ex.get("text_a_is_closer", False))
        }

        fout.write(json.dumps(out, ensure_ascii=False) + "\n")
PY
'


                                                                            MB  
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.8.20). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/auth/__init__.py:52: FutureWarning: 
    You are using a Python version 3.8 past its end of life. Google will update
    google-auth with critical bug fixes on a best-effort basis, but not
    with any other fixes or features. Please upgrade your Python version,
    and then update google-auth.
    
  warnings.warn(eol_message.format("3.8"), FutureWarning)
/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/oauth2/__init__.py:38: FutureWarning: 
    You are using a Python v

### Test pentru NLP

In [ ]:
%%bash
set -e

./bin/micromamba run -n allennlp38 bash -lc '
set -e
export LD_LIBRARY_PATH="$CONDA_PREFIX/lib:$LD_LIBRARY_PATH"
export LD_PRELOAD="$CONDA_PREFIX/lib/libstdc++.so.6:$CONDA_PREFIX/lib/libgcc_s.so.1"
export PYTHONUNBUFFERED=1
export TOKENIZERS_PARALLELISM=false

python -u - <<PY
import re, sys, pathlib
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

ANCHOR_PATH = "anchor.txt"
A_PATH = "a.txt"
B_PATH = "b.txt"

SRL_URL = "https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz"
srl = Predictor.from_path(SRL_URL, cuda_device=-1)

ARG_RE = re.compile(r"\\[([A-Z0-9\\-]+): ([^\\]]+)\\]")

def normalize_text(text: str) -> str:
    text = str(text).replace("\\u00a0", " ")
    text = " ".join(text.split())
    return text

def split_sents(text: str):
    text = normalize_text(text)
    if not text:
        return []

    return [p.strip() for p in re.split(r"(?<=[\\.\\!\\?\\;])\\s+", text) if p.strip()]

def parse(desc: str):
    return {k: v.strip() for k, v in ARG_RE.findall(desc)}

STOP_VERBS = {
    "be","is","are","was","were","been","being",
    "have","has","had","do","does","did",
    "will","would","can","could","should","may","might","must"
}

def srl_events(text, keep=("ARG0","ARG1","ARG2"), min_args=1):
    events = []
    for sent in split_sents(text):
        out = srl.predict(sentence=sent)
        for fr in out.get("verbs", []):
            d = parse(fr.get("description",""))
            verb = (fr.get("verb") or d.get("V","")).strip().lower()
            if not verb or verb in STOP_VERBS:
                continue

            args = [d[a].lower() for a in keep if a in d]
            if len(args) < min_args:
                continue

            events.append(" | ".join([verb] + args))
    return events

def load_text(path: str) -> str:
    p = pathlib.Path(path)
    if not p.exists():
        sys.exit(f"[ERROR] File not found: {path}")
    return p.read_text(encoding="utf-8").strip()

def inspect_text(title: str, text: str):
    print("\\n" + "="*100)
    print(title)
    print("="*100)

    text = normalize_text(text)
    if not text:
        print("(empty)")
        return

    sents = split_sents(text)
    print(f"[SENTENCES] {len(sents)}")
    for i, sent in enumerate(sents, 1):
        print(f"{i:02d}. {sent}")

    print("\\n[SRL RAW]")
    for i, sent in enumerate(sents, 1):
        out = srl.predict(sentence=sent)
        verbs = out.get("verbs", [])
        print(f"\\nSentence {i:02d}: {sent}")
        if not verbs:
            print("  (no verb frames)")
            continue
        for j, fr in enumerate(verbs, 1):
            v = fr.get("verb","")
            desc = fr.get("description","")
            print(f"  Frame {j:02d} | verb={v}")
            print(f"    {desc}")

    print("\\n[EVENTS extracted] (ARG0, ARG1, ARG2)")
    ev = srl_events(text)
    if not ev:
        print("  (no events)")
    else:
        for i, e in enumerate(ev, 1):
            print(f"  {i:02d}. {e}")

anchor = load_text(ANCHOR_PATH)
a_text = load_text(A_PATH)
b_text = load_text(B_PATH)

inspect_text("ANCHOR", anchor)
inspect_text("A", a_text)
inspect_text("B", b_text)
PY
'


ANCHOR
[SENTENCES] 3
01. Having been a victim of bullying, life at his new school does not take a turn to the better for Leo Weiß (played by Torge Oelrich (de).
02. In order to no longer be the misfit and to exercise power over his fellow students, he secretly infects them with a zombie virus.
03. As he is the only one who knows that a training in dancing, handicrafts or mathematics can transform them back to normal, Leo soon becomes the school's highly acclaimed hero.

[SRL RAW]

Sentence 01: Having been a victim of bullying, life at his new school does not take a turn to the better for Leo Weiß (played by Torge Oelrich (de).
  Frame 01 | verb=Having
    [V: Having] been a victim of bullying , life at his new school does not take a turn to the better for Leo Weiß ( played by Torge Oelrich ( de ) .
  Frame 02 | verb=been
    Having [V: been] [ARG2: a victim of bullying] , life at his new school does not take a turn to the better for Leo Weiß ( played by Torge Oelrich ( de ) .
  Frame 

/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.8.20). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/auth/__init__.py:52: FutureWarning: 
    You are using a Python version 3.8 past its end of life. Google will update
    google-auth with critical bug fixes on a best-effort basis, but not
    with any other fixes or features. Please upgrade your Python version,
    and then update google-auth.
    
  warnings.warn(eol_message.format("3.8"), FutureWarning)
/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/oauth2/__init__.py:38: FutureWarning: 
    You are using a Python v

### Demo

In [ ]:
%%bash
set -e

./bin/micromamba run -n allennlp38 python - <<'PY'
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

SRL_URL = "https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz"
predictor = Predictor.from_path(SRL_URL, cuda_device=-1)

sentence = "Dan found the purse and returned it to Anna."
out = predictor.predict(sentence=sentence)

print("Words:", out["words"])
print("\nFrames:")
for v in out["verbs"]:
    print(v["verb"], "=>", v["description"])
PY


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Words: ['Dan', 'found', 'the', 'purse', 'and', 'returned', 'it', 'to', 'Anna', '.']

Frames:
found => [ARG0: Dan] [V: found] [ARG1: the purse] and returned it to Anna .
returned => [ARG0: Dan] found the purse and [V: returned] [ARG1: it] [ARG2: to Anna] .


/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.8.20). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/auth/__init__.py:52: FutureWarning: 
    You are using a Python version 3.8 past its end of life. Google will update
    google-auth with critical bug fixes on a best-effort basis, but not
    with any other fixes or features. Please upgrade your Python version,
    and then update google-auth.
    
  warnings.warn(eol_message.format("3.8"), FutureWarning)
/root/.local/share/mamba/envs/allennlp38/lib/python3.8/site-packages/google/oauth2/__init__.py:38: FutureWarning: 
    You are using a Python v